In [ ]:
import os
import pandas as pd

from geoscripts.dz import dz

In [ ]:
directory = 'proc_no_filter'
isoplotr_dir = 'isoplotr_raw_no_filter'
smps = []

os.makedirs(isoplotr_dir,exist_ok=True)

nickname_dic = {'CT15076':'KZ1','CT15082':'KZ2','CT15092':'KZ3','CT15127':'KZ4','CT15099':'KZ5','CT15113':'KZ6','AB0913':'RU1','CT130918-2A':'SV1',
'100211-9A':'SV2','CT130918-9A':'SV3','CT130919-5A':'SV4','100411-5':'SV5','CT130919-8A':'SV6','AB0926':'SV7','100211-1A':'SV8'}

for file in os.listdir(directory):
    filepath = os.path.join(directory,file)
    data = pd.read_csv(filepath,index_col=0)
    
    name = file[:-4]
    
    smp = dz.DZSample(name=name,agedata=data)
    smp.nickname = nickname_dic[smp.name]

    print(smp.name,smp.nickname)

    # Set up new df in IsoplotR format (1se abs)
    working_table = pd.DataFrame([],columns=['Pb207U235','errPb207U235','Pb206U238','errPb206U238'])
   
    # Flip ratios
    working_table['Pb207U235'] = smp.agedata['207Pb/235U']
    working_table['Pb206U238'] = smp.agedata['206Pb/238U']

    # Convert 2s percentage errors to 1s absolute errors
    working_table['errPb207U235'] =  working_table['Pb207U235']*(smp.agedata['207Pb/235U_err']/100)/2
    working_table['errPb206U238'] = working_table['Pb206U238']*(smp.agedata['206Pb/238U_err']/100)/2

    filename = smp.name + '.csv'

    output_dir = os.path.join(isoplotr_dir,filename)

    working_table.to_csv(output_dir,index=None)

    smp.save(path='dz_no_filter/')

In [ ]:
directory = 'proc_rel_age_filter'
isoplotr_dir = 'isoplotr_rel_age_filter'

os.makedirs(isoplotr_dir,exist_ok=True)

smps_new = dz.load_all('dz_rel_age_filter/')

for smp in smps:
    output = pd.DataFrame([],columns=['tbest','err'])
    output['tbest'] = smp.bestage
    
    # Convert 2s errors to 1s
    output['err'] = smp.besterror/2

    output.to_csv(isoplotr_dir + '/' + smp.name + '.csv',index=False)